In [1]:
import os
import cv2 , math
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [2]:
#color detection
def detect_card_color(img):
    """ Determining the color of the card using HSV color space """
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # color ranges for UNO colors
    colors = {
        "Red": ([0, 120, 70], [10, 255, 255]),
        "Yellow": ([20, 100, 100], [30, 255, 255]),
        "Green": ([35, 100, 100], [85, 255, 255]),
        "Blue": ([90, 100, 100], [130, 255, 255])
    }

    for color, (lower, upper) in colors.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        if cv2.countNonZero(mask) > 500:  # If enough pixels match the color
            return color

    return "unknown"


In [3]:
def live_camera(img):
    # Load trained model
    model = tf.keras.models.load_model("uno_number_detection.keras")
    
    # Define categories
    categories = ["Number_1", "Number_2", "Number_3","Number_4","Number_5","Number_6","Number_7","Number_8","Number_9","Symbol_Draw_2","Symbol_Reverse","Symbol_Skip"]
    
       
    frame=img.copy()
    
    img_size=224

    #preprocessing cycle -1 :
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)#converting to gray scale
    _, thresh = cv2.threshold(gray,200, 255, cv2.THRESH_BINARY) #thresholding the image
    edges = cv2.Canny(thresh, 100, 200)# edge detection

    kernel = np.ones((5, 5), np.uint8) #matrix size for dilation and eroding 5 x 5

    dilated = cv2.dilate(edges, kernel, iterations=2) #dilation
    eroded = cv2.erode(dilated, kernel, iterations=1) #erosion
    
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) # finds the contour
    
    cv2.drawContours(frame, contours, -1, (0, 255, 0), 3) # draw a contour around the uno card
    
    img_input=frame.copy()
    copy_image=frame.copy()
    
    for contour in contours:
        rect = cv2.minAreaRect(contour)

        box = cv2.boxPoints(rect) # creating the box points
        box = box.astype(int) # getting the x, y value for each cornor points
        
        cv2.drawContours(img_input, [box], -1, (255, 0, 0), 3) # drawing a box

    #calculating the center point
    center_x = int(np.mean(box[:, 0]))
    center_y = int(np.mean(box[:, 1]))

    # Draw the center point
    cv2.circle(img_input, (center_x, center_y), 5, (0, 0, 255), -1)

    pt1, pt2, pt3, pt4 = box #four points of the rectangle

    # midpoint of pt1 - pt2
    mid1 = ((pt1[0] + pt2[0]) // 2, (pt1[1] + pt2[1]) // 2)
    
    # midpoint of pt3 - pt4
    mid2 = ((pt3[0] + pt4[0]) // 2, (pt3[1] + pt4[1]) // 2)
    
    #midpoint of pt1 - pt4
    mid3=((pt1[0] + pt4[0]) // 2, (pt1[1] + pt4[1]) // 2)
    
    #midpoint of pt2 - pt3
    mid4 = ((pt3[0] + pt2[0]) // 2, (pt3[1] + pt2[1]) // 2)

    # Draw the line between midpoints
    cv2.line(img_input, mid1, mid2, (255, 255, 0), 2)  
    cv2.line(img_input, mid3, mid4, (0, 255, 0), 2)
    
    #distance between points or length of the line
    line1_length=math.sqrt((mid2[0] - mid1[0])**2+(mid2[1] - mid1[1])**2)
    line2_length=math.sqrt((mid4[0] - mid3[0])**2+(mid4[1] - mid3[1])**2)

    #finding the longest line and setting those mid point to take angle
    if line1_length>line2_length:
        midpoint1=mid1
        midpoint2=mid2
    else:
        midpoint1=mid3
        midpoint2=mid4
        
    # length of reference line
    line_length = 100
    
    # Starting point is center
    start_point = (center_x, center_y)
    
    # End point straight up (90 degrees)
    end_point = (center_x, center_y - line_length)
    
    # Drawing the vertical reference line
    cv2.line(img_input, start_point, end_point, (0, 255, 255), 2)  # Yellow line
                    
    # Vector of midpoint line
    vector_x = midpoint2[0] - midpoint1[0]
    vector_y = midpoint2[1] - midpoint1[1]
    
    # Angle between vector and vertical line (0, -1)
    angle_rad = np.arctan2(vector_y, vector_x) - np.arctan2(-1, 0)
    angle_deg = np.degrees(angle_rad)
    angle_deg = angle_deg % 360 
                    
    # Getting image center
    (h, w) = img_input.shape[:2]
    image_center = (w // 2, h // 2)
    
    # rotation matrix
    M = cv2.getRotationMatrix2D(image_center, angle_deg, 1.0)
    
    # Perform the rotation
    rotated = cv2.warpAffine(copy_image, M, (w, h))
    #show_image(rotated, "Rotated image")
    
    # preprocessing - cycle2

    gray_2 = cv2.cvtColor(rotated, cv2.COLOR_BGR2GRAY) 
    _, thresh_2 = cv2.threshold(gray_2,220, 255, cv2.THRESH_BINARY)             
    edges_2 = cv2.Canny(thresh_2, 100, 200)
                  
    kernel = np.ones((5, 5), np.uint8)
    
    dilated_2 = cv2.dilate(edges_2, kernel, iterations=2)
    eroded_2 = cv2.erode(dilated_2, kernel, iterations=1)
    
    contours, _ = cv2.findContours(eroded_2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Process only if we have at least one valid contour
    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(rotated, (x, y), (x + w, y + h), (0, 255, 0), 3)
        
        
        if area > 500:  # Filter small areas
                                   
            cropped = rotated[y:y+h, x:x+w]                       
            resize_img = cv2.resize(cropped, (224, 224))
            
            # preprocessing - cycle3
            
            gray_3 = cv2.cvtColor(resize_img, cv2.COLOR_BGR2GRAY)                       
            _, thresh_3 = cv2.threshold(gray_3,220, 255, cv2.THRESH_BINARY)
            edges_3 = cv2.Canny(thresh_3, 100, 200)
            
            kernel = np.ones((5, 5), np.uint8)
            
            dilated_3 = cv2.dilate(edges_3, kernel, iterations=2)
            eroded_3 = cv2.erode(dilated_3, kernel, iterations=1)
            
            contours, _ = cv2.findContours(thresh_3, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

            for cnt in contours:
                area2 = cv2.contourArea(cnt)
                
                
                if area2> 19000 and area2 <30000:
                                        
                    background = np.ones((224, 224, 3), dtype=np.uint8) * 255  
                    
                    # Creating mask for the oval contour
                    mask = np.zeros_like(resize_img)
                    cv2.drawContours(mask, [cnt], 0, (255, 255, 255), -1)
                                                   
                    # Applying mask to extract oval region
                    masked = cv2.bitwise_and(resize_img, mask)
                    
                    # Crop the contour region
                    x, y, w, h = cv2.boundingRect(cnt)
                    cropped = masked[y:y+h, x:x+w]
                    
                    # Get shape of cropped region
                    ch, cw = cropped.shape[:2]
    
                    target_size = 224
                    # Compute offsets to center it
                    x_offset = (target_size - cw) // 2
                    y_offset = (target_size - ch) // 2
                    
                    # Pasting cropped oval into the center of white background
                    background[y_offset:y_offset+ch, x_offset:x_offset+cw] = cropped
    
                    #final_crop=cv2.cvtColor(background, cv2.COLOR_BGR2RGB)
                    gray_final = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
                    _, thresh_final = cv2.threshold(gray_final,226, 255, cv2.THRESH_BINARY)
                    #show_image(thresh_final,"final - image",cmap="grey")

                    # Resize to final model input size
                    final_img = cv2.resize(thresh_final, (img_size, img_size))
                    """
                    # Preprocess the image
                    img_processed = preprocess_input(final_img.astype(np.float32))
                    img_array = np.expand_dims(img_processed, axis=0)

                    predictions = model.predict(img_array, verbose=0)
                    class_id = np.argmax(predictions)
                    confidence = np.max(predictions)

                    label = f"{categories[class_id]}"

                    color = detect_card_color(background)

                #cv2.putText(frame,"Label -"+label,(10, 30),cv2.FONT_HERSHEY_SIMPLEX,1,(0, 255, 0),1,cv2.LINE_AA)
                #cv2.putText(frame,"Color -"+color,(30, 60),cv2.FONT_HERSHEY_SIMPLEX,1,(0, 255, 0),1,cv2.LINE_AA)
                """
                
    return frame,final_img,background




In [ ]:
# Form implementation generated from reading ui file 'gui.ui'
#
# Created by: PyQt6 UI code generator 6.4.2
#
# WARNING: Any manual changes made to this file will be lost when pyuic6 is
# run again.  Do not edit this file unless you know what you are doing.


from PyQt6 import QtCore, QtGui, QtWidgets
from PyQt6.QtWidgets import QFileDialog


    
class Ui_Dialog(object):
    def setupUi(self, Dialog):
        Dialog.setObjectName("Dialog")
        Dialog.resize(640, 480)
        self.buttonBox = QtWidgets.QDialogButtonBox(parent=Dialog)
        self.buttonBox.setGeometry(QtCore.QRect(10, 440, 621, 32))
        self.buttonBox.setOrientation(QtCore.Qt.Orientation.Horizontal)
        self.buttonBox.setStandardButtons(QtWidgets.QDialogButtonBox.StandardButton.Cancel|QtWidgets.QDialogButtonBox.StandardButton.Ok)
        self.buttonBox.setObjectName("buttonBox")
        
        self.textBrowser = QtWidgets.QTextBrowser(parent=Dialog)
        self.textBrowser.setGeometry(QtCore.QRect(20, 20, 601, 111))
        self.textBrowser.setObjectName("textBrowser")
        
        self.textEdit = QtWidgets.QTextEdit(parent=Dialog)
        self.textEdit.setGeometry(QtCore.QRect(20, 140, 291, 291))
        self.textEdit.setObjectName("textEdit")
        
        self.camera_number = QtWidgets.QSpinBox(parent=Dialog)
        self.camera_number.setGeometry(QtCore.QRect(180, 190, 121, 41))
        self.camera_number.setMaximum(2)
        self.camera_number.setObjectName("camera_number") #camera index number
        
        self.textBrowser_2 = QtWidgets.QTextBrowser(parent=Dialog)
        self.textBrowser_2.setGeometry(QtCore.QRect(30, 190, 131, 41))
        self.textBrowser_2.setObjectName("textBrowser_2")
        
        self.Live_start = QtWidgets.QPushButton(parent=Dialog)
        self.Live_start.setGeometry(QtCore.QRect(30, 240, 131, 41))
        self.Live_start.setObjectName("Live_start") #start camera
        
        self.Live_stop = QtWidgets.QPushButton(parent=Dialog)
        self.Live_stop.setGeometry(QtCore.QRect(180, 240, 121, 41))
        self.Live_stop.setObjectName("Live_stop") #stop camera
        
        self.textEdit_2 = QtWidgets.QTextEdit(parent=Dialog)
        self.textEdit_2.setGeometry(QtCore.QRect(330, 140, 291, 291))
        self.textEdit_2.setObjectName("textEdit_2")
        
        self.Imagepredict = QtWidgets.QPushButton(parent=Dialog)
        self.Imagepredict.setGeometry(QtCore.QRect(490, 260, 111, 41))
        self.Imagepredict.setObjectName("Imagepredict") #image predict button
        
        self.textBrowser_3 = QtWidgets.QTextBrowser(parent=Dialog)
        self.textBrowser_3.setGeometry(QtCore.QRect(340, 190, 261, 61))
        self.textBrowser_3.setObjectName("textBrowser_3")

        """
        self.FileNumberinput = QtWidgets.QTextEdit(parent=Dialog)
        self.FileNumberinput.setGeometry(QtCore.QRect(340, 260, 141, 41))
        font = QtGui.QFont()
        font.setFamily("URW Bookman")
        font.setPointSize(11)
        font.setBold(True)
        self.FileNumberinput.setFont(font)
        self.FileNumberinput.setObjectName("FileNumberinput") # File name as number input
        """
        self.Live_predictionoutput = QtWidgets.QTextBrowser(parent=Dialog)
        self.Live_predictionoutput.setGeometry(QtCore.QRect(40, 320, 251, 91))
        self.Live_predictionoutput.setObjectName("Live_predictionoutput") # live predict output
        
        self.label = QtWidgets.QLabel(parent=Dialog)
        self.label.setGeometry(QtCore.QRect(40, 291, 251, 31))
        self.label.setObjectName("label")
        self.Imagepredictionoutput = QtWidgets.QTextBrowser(parent=Dialog)
        self.Imagepredictionoutput.setGeometry(QtCore.QRect(340, 329, 261, 91))
        self.Imagepredictionoutput.setObjectName("Imagepredictionoutput") # image predict output
        
        self.Label1 = QtWidgets.QLabel(parent=Dialog)
        self.Label1.setGeometry(QtCore.QRect(340, 300, 261, 31))
        self.Label1.setObjectName("Label1")
        self.textBrowser_4 = QtWidgets.QTextBrowser(parent=Dialog)
        self.textBrowser_4.setGeometry(QtCore.QRect(30, 140, 271, 41))
        self.textBrowser_4.setObjectName("textBrowser_4")
        self.textBrowser_5 = QtWidgets.QTextBrowser(parent=Dialog)
        self.textBrowser_5.setGeometry(QtCore.QRect(340, 140, 271, 41))
        self.textBrowser_5.setObjectName("textBrowser_5")
        self.textEdit_2.raise_()
        self.buttonBox.raise_()
        self.textBrowser.raise_()
        self.textEdit.raise_()
        self.camera_number.raise_()
        self.textBrowser_2.raise_()
        self.Live_start.raise_()
        self.Live_stop.raise_()
        self.Imagepredict.raise_()
        self.textBrowser_3.raise_()
        #self.FileNumberinput.raise_()
        self.Live_predictionoutput.raise_()
        self.label.raise_()
        self.Imagepredictionoutput.raise_()
        self.Label1.raise_()
        self.textBrowser_4.raise_()
        self.textBrowser_5.raise_()

        self.retranslateUi(Dialog)
        self.buttonBox.accepted.connect(Dialog.accept) # type: ignore
        self.buttonBox.rejected.connect(Dialog.reject) # type: ignore
        self.Imagepredict.clicked.connect(self.predict_from_file)
        
        self.Live_start.clicked.connect(self.start_camera)
        self.Live_stop.clicked.connect(self.stop_camera)

        self.cap = None
        self.timer = QtCore.QTimer()
        self.timer.timeout.connect(self.update_frame)
        
        QtCore.QMetaObject.connectSlotsByName(Dialog)
        

    #program to run the camera and detect
    def start_camera(self):
        camera_index = self.camera_number.value()
        self.cap = cv2.VideoCapture(camera_index)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        self.timer.start(30)

    def update_frame(self):
        ret, frame = self.cap.read()
        if ret:
            processed_frame,imageforprediction,imageforcolor = live_camera(frame)

             # Load trained model
            model = tf.keras.models.load_model("uno_number_detection.keras")
    
            # Define categories
            categories = ["Number_1", "Number_2", "Number_3","Number_4","Number_5","Number_6","Number_7","Number_8","Number_9","Symbol_Draw_2","Symbol_Reverse","Symbol_Skip"]
            
            #self.Live_predictionoutput.setText("Live prediction:")
            # Preprocess the image
            img_processed = preprocess_input(imageforprediction.astype(np.float32))
            img_array = np.expand_dims(img_processed, axis=0)

            predictions = model.predict(img_array, verbose=0)
            class_id = np.argmax(predictions)
            confidence = np.max(predictions)

            label = f"{categories[class_id]}"

            color = detect_card_color(imageforcolor)

            cv2.putText(processed_frame,"Label -"+label,(10, 30),cv2.FONT_HERSHEY_SIMPLEX,1,(0, 255, 0),1,cv2.LINE_AA)
            cv2.putText(processed_frame,"Color -"+color,(30, 60),cv2.FONT_HERSHEY_SIMPLEX,1,(0, 255, 0),1,cv2.LINE_AA)
            
            self.Live_predictionoutput.setText(label+"\n" + color)
            
            cv2.imshow("Live Detection", processed_frame)
            cv2.waitKey(1)

    def stop_camera(self):
        self.timer.stop()
        if self.cap:
            self.cap.release()
            self.cap = None
        cv2.destroyAllWindows()

    def predict_from_file(self):
        file_path, _ = QFileDialog.getOpenFileName(None, "Select Image", "", "Image Files (*.png *.jpg *.jpeg *.bmp)") #took it from google
        if file_path:
            image = cv2.imread(file_path)
            processed_frame, imageforprediction, imageforcolor = live_camera(image)
            
            # Load trained model
            model = tf.keras.models.load_model("uno_number_detection.keras")
    
            # Define categories
            categories = ["Number_1", "Number_2", "Number_3","Number_4","Number_5","Number_6","Number_7","Number_8","Number_9","Symbol_Draw_2","Symbol_Reverse","Symbol_Skip"]
            
            #self.Live_predictionoutput.setText("Live prediction:")
            # Preprocess the image
            img_processed = preprocess_input(imageforprediction.astype(np.float32))
            img_array = np.expand_dims(img_processed, axis=0)

            predictions = model.predict(img_array, verbose=0)
            class_id = np.argmax(predictions)
            confidence = np.max(predictions)

            label = f"{categories[class_id]}"

            color = detect_card_color(imageforcolor)

            cv2.putText(processed_frame,"Label -"+label,(10, 30),cv2.FONT_HERSHEY_SIMPLEX,1,(0, 255, 0),1,cv2.LINE_AA)
            cv2.putText(processed_frame,"Color -"+color,(30, 60),cv2.FONT_HERSHEY_SIMPLEX,1,(0, 255, 0),1,cv2.LINE_AA)
            
            self.Imagepredictionoutput.setText(label+"\n" + color)
            
            cv2.imshow("Image prediction output", processed_frame)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            
    def retranslateUi(self, Dialog):
        _translate = QtCore.QCoreApplication.translate
        Dialog.setWindowTitle(_translate("Dialog", "Dialog"))
        self.textBrowser.setHtml(_translate("Dialog", "<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0//EN\" \"http://www.w3.org/TR/REC-html40/strict.dtd\">\n"
"<html><head><meta name=\"qrichtext\" content=\"1\" /><meta charset=\"utf-8\" /><style type=\"text/css\">\n"
"p, li { white-space: pre-wrap; }\n"
"hr { height: 1px; border-width: 0; }\n"
"li.unchecked::marker { content: \"\\2610\"; }\n"
"li.checked::marker { content: \"\\2612\"; }\n"
"</style></head><body style=\" font-family:\'Segoe UI\'; font-size:12.75pt; font-weight:400; font-style:normal;\">\n"
"<p align=\"center\" style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\"><span style=\" font-family:\'Ubuntu\'; font-size:11pt; font-weight:600;\">PDE4434 - IntelligentSensing</span></p>\n"
"<p align=\"center\" style=\"-qt-paragraph-type:empty; margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px; font-family:\'Ubuntu\'; font-size:11pt; font-weight:600;\"><br /></p>\n"
"<p align=\"center\" style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\"><span style=\" font-family:\'Ubuntu\'; font-size:11pt;\">Coursework 2</span></p>\n"
"<p align=\"center\" style=\"-qt-paragraph-type:empty; margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px; font-family:\'Ubuntu\'; font-size:11pt;\"><br /></p>\n"
"<p align=\"center\" style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\"><span style=\" font-family:\'Ubuntu\'; font-size:11pt; font-weight:600; color:#204a87;\">Uno Card Color and Number Detection </span></p>\n"
"<p style=\"-qt-paragraph-type:empty; margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px; font-family:\'Ubuntu\'; font-size:11pt;\"><br /></p></body></html>"))
        self.textEdit.setHtml(_translate("Dialog", "<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0//EN\" \"http://www.w3.org/TR/REC-html40/strict.dtd\">\n"
"<html><head><meta name=\"qrichtext\" content=\"1\" /><meta charset=\"utf-8\" /><style type=\"text/css\">\n"
"p, li { white-space: pre-wrap; }\n"
"hr { height: 1px; border-width: 0; }\n"
"li.unchecked::marker { content: \"\\2610\"; }\n"
"li.checked::marker { content: \"\\2612\"; }\n"
"</style></head><body style=\" font-family:\'Segoe UI\'; font-size:12.75pt; font-weight:400; font-style:normal;\">\n"
"<p align=\"center\" style=\"-qt-paragraph-type:empty; margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\"><br /></p></body></html>"))
        self.textBrowser_2.setHtml(_translate("Dialog", "<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0//EN\" \"http://www.w3.org/TR/REC-html40/strict.dtd\">\n"
"<html><head><meta name=\"qrichtext\" content=\"1\" /><meta charset=\"utf-8\" /><style type=\"text/css\">\n"
"p, li { white-space: pre-wrap; }\n"
"hr { height: 1px; border-width: 0; }\n"
"li.unchecked::marker { content: \"\\2610\"; }\n"
"li.checked::marker { content: \"\\2612\"; }\n"
"</style></head><body style=\" font-family:\'Segoe UI\'; font-size:12.75pt; font-weight:400; font-style:normal;\">\n"
"<p style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\">Camera Index</p></body></html>"))
        self.Live_start.setText(_translate("Dialog", "Start"))
        self.Live_stop.setText(_translate("Dialog", "Stop"))
        self.textEdit_2.setHtml(_translate("Dialog", "<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0//EN\" \"http://www.w3.org/TR/REC-html40/strict.dtd\">\n"
"<html><head><meta name=\"qrichtext\" content=\"1\" /><meta charset=\"utf-8\" /><style type=\"text/css\">\n"
"p, li { white-space: pre-wrap; }\n"
"hr { height: 1px; border-width: 0; }\n"
"li.unchecked::marker { content: \"\\2610\"; }\n"
"li.checked::marker { content: \"\\2612\"; }\n"
"</style></head><body style=\" font-family:\'Segoe UI\'; font-size:12.75pt; font-weight:400; font-style:normal;\">\n"
"<p align=\"center\" style=\"-qt-paragraph-type:empty; margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\"><br /></p></body></html>"))
        self.Imagepredict.setText(_translate("Dialog", "Predict"))
        self.textBrowser_3.setHtml(_translate("Dialog", "<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0//EN\" \"http://www.w3.org/TR/REC-html40/strict.dtd\">\n"
"<html><head><meta name=\"qrichtext\" content=\"1\" /><meta charset=\"utf-8\" /><style type=\"text/css\">\n"
"p, li { white-space: pre-wrap; }\n"
"hr { height: 1px; border-width: 0; }\n"
"li.unchecked::marker { content: \"\\2610\"; }\n"
"li.checked::marker { content: \"\\2612\"; }\n"
"</style></head><body style=\" font-family:\'Segoe UI\'; font-size:12.75pt; font-weight:400; font-style:normal;\">\n"
"<p align=\"center\" style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\">File name in number</p>\n"
"<p align=\"center\" style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\">(1 to 30)</p></body></html>"))
        self.label.setText(_translate("Dialog", "Detected Card :"))
        self.Label1.setText(_translate("Dialog", "Detected Card :"))
        self.textBrowser_4.setHtml(_translate("Dialog", "<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0//EN\" \"http://www.w3.org/TR/REC-html40/strict.dtd\">\n"
"<html><head><meta name=\"qrichtext\" content=\"1\" /><meta charset=\"utf-8\" /><style type=\"text/css\">\n"
"p, li { white-space: pre-wrap; }\n"
"hr { height: 1px; border-width: 0; }\n"
"li.unchecked::marker { content: \"\\2610\"; }\n"
"li.checked::marker { content: \"\\2612\"; }\n"
"</style></head><body style=\" font-family:\'Segoe UI\'; font-size:12.75pt; font-weight:400; font-style:normal;\">\n"
"<p align=\"center\" style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\"><span style=\" font-size:14pt; font-weight:700; color:#ff0000;\">Live Camera Detection</span></p></body></html>"))
        self.textBrowser_5.setHtml(_translate("Dialog", "<!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.0//EN\" \"http://www.w3.org/TR/REC-html40/strict.dtd\">\n"
"<html><head><meta name=\"qrichtext\" content=\"1\" /><meta charset=\"utf-8\" /><style type=\"text/css\">\n"
"p, li { white-space: pre-wrap; }\n"
"hr { height: 1px; border-width: 0; }\n"
"li.unchecked::marker { content: \"\\2610\"; }\n"
"li.checked::marker { content: \"\\2612\"; }\n"
"</style></head><body style=\" font-family:\'Segoe UI\'; font-size:12.75pt; font-weight:400; font-style:normal;\">\n"
"<p style=\" margin-top:0px; margin-bottom:0px; margin-left:0px; margin-right:0px; -qt-block-indent:0; text-indent:0px;\"><span style=\" font-size:14pt; font-weight:700; color:#00007f;\">Testing images from dataset</span></p></body></html>"))


if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    Dialog = QtWidgets.QDialog()
    ui = Ui_Dialog()
    ui.setupUi(Dialog)
    Dialog.show()
    sys.exit(app.exec())
